In [46]:
# load data and split into train and test sets
import torch
from load_data import sim_arr
from sklearn.model_selection import train_test_split

# reshape from batch, height, width, channel, to batch, channel, height, width
sim_arr_transformed = sim_arr.reshape(
    sim_arr.shape[0], sim_arr.shape[3], sim_arr.shape[1], sim_arr.shape[2])
train_set, test_set = train_test_split(
    sim_arr_transformed, test_size=0.2, random_state=42)

# convert to tensor
train_set = torch.tensor(train_set, dtype=torch.float32)
test_set = torch.tensor(test_set, dtype=torch.float32)

In [47]:
from AE_torch import Autoencoder
from search_space import search_space
import optuna
import warnings


warnings.filterwarnings("ignore")


def objective(trial):
    #clear clutter from previous runs
    torch.cuda.empty_cache()

    # define search space
    num_layers, poolsize, channels, kernel_sizes, dilations, activations = search_space(trial, input_dim=3, output_dim=3)

    # define model
    model = Autoencoder(num_layers=num_layers,
                        poolsize=poolsize,
                        channels=channels,
                        kernel_sizes=kernel_sizes,
                        dilations=dilations,
                        activations=activations,
                        epochs=10,
                        batch_size=32,
                        learning_rate=1e-3,
                        data=train_set)
    
    # train model with k-fold cross validation
    val_losses = model.cross_val()
    val_loss = sum(val_losses) / len(val_losses)
    print("Validation loss:", val_loss)
    return val_loss

# delete the study
#optuna.delete_study(study_name="autoencoder_torch1", storage="sqlite:///autoencoder.db")

# define study
study = optuna.create_study(direction="minimize",
                            pruner=optuna.pruners.HyperbandPruner(),
                            study_name="autoencoder_torch1",
                            storage="sqlite:///autoencoder.db",
                            load_if_exists=True)

study.optimize(objective, n_trials=0)

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", str(best_params))
print("Best value:", study.best_value)


[I 2024-05-08 13:43:46,936] Using an existing study with name 'autoencoder_torch1' instead of creating a new one.


Best hyperparameters: {'num_layers': 3, 'poolsize_3': [2, 20, 2], 'channels_0': 9, 'channels_1': 10, 'kernel_size_0': 8, 'kernel_size_1': 20, 'kernel_size_2': 5, 'dilation_0': 1, 'dilation_1': 2, 'dilation_2': 3, 'activation_0': 'nn.Tanh', 'activation_1': 'nn.SiLU', 'activation_2': 'nn.Tanh'}
Best value: 0.02036499066546983


In [48]:
from sklearn.model_selection import KFold
from sklearn.cluster import DBSCAN

class DBSCANLayer(torch.nn.Module):
    """DBSCAN layer that inherits from PyTorch's nn.Module class."""

    def __init__(self, eps, min_samples):
        super(DBSCANLayer, self).__init__()
        self.eps = eps
        self.min_samples = min_samples

    def forward(self, x):
        """Forward pass through the DBSCAN layer."""
        # Convert PyTorch tensor to numpy array
        x_cpu = torch.cat(x).cpu().detach().numpy()


        # Perform DBSCAN clustering
        dbscan = DBSCAN(eps=self.eps, min_samples=self.min_samples)
        clusters = dbscan.fit_predict(x_cpu)

        # Convert clusters to PyTorch tensor
        clusters_tensor = torch.from_numpy(clusters).to(x.device)
        return clusters_tensor

class JointAutoencoder(torch.nn.Module):
    def __init__(self, encoder, decoder):
        super(JointAutoencoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cluster = cluster
        self.epochs = 10
        self.batch_size = 32
        self.data = train_set
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)

    def forward(self, x):
        x = self.encoder(x)
        clusters = self.cluster(x)
        x = self.decoder(x)
        
        return x, clusters
    
    def _get_loss(self, x):
        x_hat, clusters = self(x)
        reconstruction_loss = torch.nn.functional.mse_loss(x_hat, x, reduction='mean')
        cluster_loss = torch.nn.functional.kl_div(clusters, torch.ones_like(clusters), reduction='mean')
        return reconstruction_loss + cluster_loss

    
    def train_model(self, dataloader, device=torch.device('cuda:0')):
        """Train the autoencoder."""
        self.to(device)
        self.train()
        for epoch in range(self.epochs):
            running_loss = 0.0
            for batch in dataloader:
                batch = batch.to(device)
                self.optimizer.zero_grad()
                loss = self._get_loss(batch)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item() * batch.size(0)
            epoch_loss = running_loss / len(dataloader)
            print(f"Epoch {epoch+1}/{self.epochs} Loss: {epoch_loss:.4f}")

    def evaluate_model(self, dataloader, device=torch.device('cuda:0')):
        """Evaluate the autoencoder."""
        self.to(device)
        self.eval()
        total_loss = 0.0
        with torch.no_grad():
            for batch in dataloader:
                batch = batch.to(device)
                loss = self._get_loss(batch)
                total_loss += loss.item() * batch.size(0)
        avg_loss = total_loss / len(dataloader)
        print(f"Validation Loss: {avg_loss:.4f}")
        return avg_loss

    def cross_val(self, n_splits=5):
        """Perform cross-validation on the autoencoder."""
        torch.backends.cudnn.benchmark = True
        kf = KFold(n_splits=n_splits, shuffle=True)
        val_losses = []
        for fold, (train_index, val_index) in enumerate(kf.split(self.data)):
            print(f"Fold {fold+1}/{n_splits}")
            train_sampler = torch.utils.data.SubsetRandomSampler(train_index)
            val_sampler = torch.utils.data.SubsetRandomSampler(val_index)

            train_loader = torch.utils.data.DataLoader(
                self.data,
                sampler=train_sampler,
                batch_size=self.batch_size,
                num_workers=12)
            val_loader = torch.utils.data.DataLoader(
                self.data,
                sampler=val_sampler,
                batch_size=self.batch_size,
                num_workers=12)

            self.train_model(train_loader)
            val_loss = self.evaluate_model(val_loader)
            val_losses.append(val_loss)
        return val_losses

# train model with best hyperparameters
num_layers, poolsize, channels, kernel_sizes, dilations, activations = search_space(
    study.best_trial, input_dim=3, output_dim=3)

model = Autoencoder(num_layers=num_layers,
                    poolsize=poolsize,
                    channels=channels,
                    kernel_sizes=kernel_sizes,
                    dilations=dilations,
                    activations=activations,
                    epochs=10,
                    batch_size=32, 
                    learning_rate=1e-3,
                    data=train_set)

cluster = DBSCANLayer(eps=0.1, min_samples=5)

joint_autoencoder = JointAutoencoder(encoder=model.encoder, decoder=model.decoder)
joint_autoencoder.cross_val()


Fold 1/5


TypeError: expected Tensor as element 1 in argument 0, but got list

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

i = np.random.randint(0, len(test_set))
in_sim = test_set[i: i + 1]
in_sim = in_sim.to('cuda:0')
out_sim = model(in_sim)
in_sim = in_sim.detach().to('cpu').numpy()
in_sim = in_sim.reshape(in_sim.shape[0], in_sim.shape[2], in_sim.shape[3], in_sim.shape[1])
out_sim = out_sim.detach().to('cpu').numpy()
out_sim = out_sim.reshape(out_sim.shape[0], out_sim.shape[2], out_sim.shape[3], out_sim.shape[1])
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(in_sim[0, ..., 2], vmin=-1, vmax=1, cmap="RdBu")
ax[1].imshow(out_sim[0, ..., 2], vmin=-1, vmax=1, cmap="RdBu")

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same